# Combining COMTRADE and Flaring data

GGFR have a global flare datasets derived from Elvidge at point locations. They further map these point locations to operators via oil field mapping. This means there are a number of datasets:

1. Point locations of flaring
2. Mapped oil field locations (from Woods Mackenzie)
3. Tabulated national accounts (of flaring)

Harshit suggested there could be an interesting combination of the COMTRADE data to see what the comparison of nationally generated flaring vs imported flaring.

Flaring Supply Index (FSI) is a concept proposed for an indicator that would be able to describe and compare the ’flaring footprint’ of different oil supply corridors from their source to the destination country’s border. The formula(s) for such an index may be described as follows.
Flare Supply Index of a Country X= (∑_(i=1)^n▒〖Crude Import by X from Country Yi * Flare Intensity of Country Yi〗)/(Total Crude Imported by Country X)


## Definitions
Flare Intensity - ???  
TOE - Tonnes of oil equivalent

## TO DO
1. TOE is not the appropriate flaring comparison for intensity, figure out what it is
2. Right now, the intensity is taken just from 2019; should use yearly data

In [37]:
import sys, os, importlib
import json, geojson, pycountry

import pandas as pd
import geopandas as gpd
import numpy as np

In [31]:
# Define input variables
# COMTRADE data can be downloaded here - https://datacatalog.worldbank.org/dataset/global-comtrade-flows-data
comtrade_file = '/home/wb411133/data/Projects/INFRA/FLOWS/Gas/GEOJSON/gas_country_flows_imports.geojson'
flare_data = '/home/public/Data/GLOBAL/INFRA/FLARING/2019_flare_catalog.csv'
flare_national_data = '/home/public/Data/GLOBAL/INFRA/FLARING/National_flaring_2019.csv'


inCom = gpd.read_file(comtrade_file)
inFlare = pd.read_csv(flare_data)
inF = pd.read_csv(flare_national_data)

The location specific flare data is not necessary for calculating the FSI, but it could be used in other calculations

In [86]:
agg = {'bcm':'sum'}
flare_combo = inFlare.groupby(['iso']).aggregate(agg).reset_index()
flare_combo.head()

,iso,bcm
0,AGO,2.325094
1,ALB,0.002380
2,ARE,1.027568
3,ARG,1.029591
4,AUS,1.405146


In [87]:
# Limit the flaring summaries to 2019
flare_2019 = inF.filter(regex="2019")
flare_2019.columns = ["Volume","Intensity"]
flare_2019['country'] = inF['country']
flare_2019.head()

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Volume,Intensity,country
0,23212.170060,5.862699884,Russia
1,17914.219000,10.37717386,Iraq
2,17293.742520,3.868390139,United States
3,13781.155540,12.59168227,Iran
4,9541.420483,29.81330551,Venezuela


In [88]:
def get_country(x):
    ''' Convert country name to ISO3
    
    :param: x [string] - name of country to convert
    :returns: [string] - ISO3 code of country
    '''
    try:
        res = pycountry.countries.search_fuzzy(x)
        return(res[0].alpha_3)
    except:
        return("NA")

#xx = get_country("Canada")
flare_2019['ISO3'] = flare_2019['country'].apply(lambda x: get_country(x))
flare_2019.head()

/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Volume,Intensity,country,ISO3
0,23212.170060,5.862699884,Russia,RUS
1,17914.219000,10.37717386,Iraq,IRQ
2,17293.742520,3.868390139,United States,USA
3,13781.155540,12.59168227,Iran,IRN
4,9541.420483,29.81330551,Venezuela,VEN


In [106]:
# Three of the country names don't grok, set them manually

#flare_2019.loc[flare_2019['ISO3'] == "NA"]
flare_2019.iloc[16, 3] = "COG"
flare_2019.iloc[30, 3] = "UAE"
flare_2019.iloc[44, 3] = "COD"
flare_2019.iloc[60, 3] = "NER"
flare_2019.loc[flare_2019['ISO3'] == "NA"]

,Volume,Intensity,country,ISO3


In [107]:
flare_2019.iloc[60]

Volume           29.8144
Intensity    10.31997231
country            Niger
ISO3                 NER
Name: 60, dtype: object

In [108]:
flare_2019.loc[flare_2019['ISO3'] == "NGA"]

,Volume,Intensity,country,ISO3
6,7825.040809,11.01746002,Nigeria,NGA


In [109]:
#Combine the flare intensity information with the comtrade data
in_flaring_combined = pd.merge(inCom, flare_2019, left_on="Partner ISO", right_on="ISO3")
#Drop unnecessary columns
in_flaring_combined = in_flaring_combined.drop(['Reporter_Pt', 'Partner_Pt', 'geometry', 'country', 'ISO3'], axis=1)

def tryFloat(x):
    try:
        return(float(x))
    except:
        return(0.)

# Multiply the TOE from the comtrade data by the intensity information
### TODO: TOE is not the unit appropriate for intensity calculations; however, the relative rank is still useful
in_flaring_combined['TOE'] = in_flaring_combined['TOE'].apply(float)
in_flaring_combined['Intensity'] = in_flaring_combined['Intensity'].apply(tryFloat)
in_flaring_combined['TOE_I'] = in_flaring_combined['TOE'] * in_flaring_combined['Intensity']
in_flaring_combined.head()

,Reporter ISO,Partner ISO,Trade Flow,Year,Qty,Trade Value (US$),TOE,Value per unit,Volume,Intensity,TOE_I
0,,AGO,Import,2017,20222000.0,7951121.0,173.14044,0.393192,2325.103979,4.400772,761.951633
1,ARG,AGO,Import,2017,23364500.0,6724342.0,200.04648,0.287802,2325.103979,4.400772,880.358986
2,AUS,AGO,Import,2016,32913124.0,9684485.0,281.80164,0.294244,2325.103979,4.400772,1240.144821
3,BEL,AGO,Import,2013,19427964.0,15754352.0,166.34190,0.810911,2325.103979,4.400772,732.032808
4,BEN,AGO,Import,2017,2519040.0,1663658.0,21.56798,0.660433,2325.103979,4.400772,94.915767


In [110]:
in_flaring_combined.to_csv("FLARING_TRADE_COMBINED.csv")

In [97]:
# Aggregate imported TOE, Value and TOE by import intensity
agg = {'TOE':'sum','Trade Value (US$)':'sum', 'Intensity':'mean','TOE_I':'sum'}
res = in_flaring_combined.groupby(['Reporter ISO', "Year"]).aggregate(agg)
res = res.reset_index()
res.head(20)

,Reporter ISO,Year,TOE,Trade Value (US$),Intensity,TOE_I
0,,2012,0.000000,1.925000e+03,0.580106,0.000000
1,,2013,2756.907202,2.774613e+08,3.393401,16073.157166
2,,2014,1.662900,1.432643e+08,32.086547,53.356715
3,,2015,0.002654,2.250000e+02,2.965364,0.007871
4,,2016,1039.645030,3.548049e+07,2.445888,3733.160229
5,,2017,158033.049028,7.077005e+09,6.928215,963669.474549
6,ABW,2012,0.000574,3.868000e+03,0.653955,0.000375
7,ABW,2013,40.373844,3.605499e+06,19.676043,794.397472
8,ABW,2014,0.029770,9.223000e+03,0.580106,0.017270
9,ABW,2015,0.182345,5.584400e+04,1.450150,0.264427


In [96]:
res.to_csv("NATIONAL_FIS_v0.csv")